<a href="https://colab.research.google.com/github/DiamondElectronicsGroup/Semi-Device-Modelling/blob/master/Semi_Device_Modelling_Ver0-2-5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# The aim for this project is to be able to simply model the characteristics of a semiconductor device given a number
# of properties of the device

In [0]:
import numpy as np
import pandas as pd
from scipy import constants as spcon
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons
plt.rcParams['figure.figsize'] = [15, 7]


%matplotlib inline
from ipywidgets import *

In [0]:
#Define the scientific constants necessary for the modelling
q = spcon.elementary_charge
perm0 = spcon.epsilon_0
kB = spcon.k

#Define the material properties relevant to this specific system
permS = 11.68
semiWF = 4.47
metalWF = 4.88
dopeConc = (1 * 10 ** 20) #in units of cm^-3
hMob = 450

#Define the device parameters, such as gate length, gate width and channel depth
lGate = 20 * 10 ** (-4)        #in units of um
wGate = 200 * 10 ** (-4)        #in units of um
dChan = 100 * 10 ** (-7)        #in units of nm

#Calculate the built in voltage for the device
vBI = (metalWF - semiWF)

#Calculate the pinch-off voltage
vP = q*((dopeConc * dChan**2) / (2 * (permS * perm0)))

#Define the system voltages
vG = np.arange(0.0, 1., 0.25)
vS = 0
vD = np.arange(-2, 8, 0.005)

#Define the input array for the current calculation
inputs = np.array([q, permS, perm0, hMob, wGate, dopeConc, dChan, lGate, vBI, vP])

In [0]:
#Calculate the depth of the depletion region in the MESFET (ie the region in which current is restricted)
#Only consider the built-in voltage
def BIDepletionDepth(inputs):
    
    q, permS, perm0, hMob, wGate, dopeConc, dChan, lGate, vBI, vP = inputs
    
    depth = np.sqrt(((2*permS*perm0)/(q*dopeConc))*(vBI))
    return depth

In [0]:
#Calculate the depth of the depletion region in the MESFET (ie the region in which current is restricted)
#Consider the built-in voltage and the gate voltage
def BIGDepletionDepth(inputs, vG):
    
    q, permS, perm0, hMob, wGate, dopeConc, dChan, lGate, vBI, vP = inputs
    
    depth = np.sqrt(((2*permS*perm0)/(q*dopeConc))*(vBI + vG))
    
    return depth

In [0]:
#Calculate the depth of the depletion region in the MESFET (ie the region in which current is restricted)
#Consider the built-in voltage, the gate voltage and the drain voltage
def TotDepletionDepth(inputs, vG, vD):
    
    q, permS, perm0, hMob, wGate, dopeConc, dChan, lGate, vBI, vP = inputs

    depth = np.sqrt(((2*permS*perm0)/(q*dopeConc))*(vBI + vG + vD))
    
    return depth

In [0]:
#Calculate the drain current based on the operating mode of the MESFET (ie taking into account vG and vD)
def DrainCurrent(inputs, vG, vD):
    
    q, permS, perm0, hMob, wGate, dopeConc, dChan, lGate, vBI, vP, = inputs
    coeff = 2 * q * hMob * wGate * dopeConc * dChan / lGate
    depthBIG = BIGDepletionDepth(inputs, vG)
    depthTot = TotDepletionDepth(inputs, vG, vD)
    
    #Define the array that will hold the values for the current
    iD = np.zeros(len(vD))
    iDSat = 0.0

    
    for i in range(len(vD)):
        #Calculate the saturation current, which is reached when vD = VDSat
        iDSat = coeff * (vG - vBI + vP - (2/3)*(vP - ((vG - vBI)/np.sqrt(vP))**(1.5)))          
        depDepth = max(depthBIG, depthTot[i])
        
        if depDepth >= dChan:
            iD[i] = 0
        
        elif vD[i] >= (vG - vBI + vP):
            iD[i] = iDSat[i]
        
        elif vD[i] < (vG - vBI + vP) and vD[i] > 0:
            GDTerm = ((vG - vBI + vD[i])/np.sqrt(vP))
            GTerm = ((vG - vBI)/np.sqrt(vP))
            iD[i] = coeff * (vD[i] - (2/3)*(GDTerm**(1.5) - GTerm**(1.5)))
            
        else:
            iD[i] = 0

    return iD, iDSat

In [9]:


@widgets.interact(
    color=['blue', 'red', 'green'], vDmax=(1., 20., 1.), vG=(0.0, 5.0, 0.25))
def plot(vG=0.50, color='blue', vDmax=10, grid=False):
    vD = np.linspace(-1., vDmax, 10000)
    fig, ax = plt.subplots(1, 1, figsize=(8, 6))
    ax.plot(vD, DrainCurrent(inputs, vG, vD)[0], color=color)
    ax.grid(grid)


interactive(children=(FloatSlider(value=0.5, description='vG', max=5.0, step=0.25), Dropdown(description='colo…